In [120]:
import numpy as np
import openrouteservice
from openrouteservice.geocode import pelias_search
import xarray as xr
from datetime import datetime
from datetime import time as dtime
from datetime import date as ddate

ORS_API_KEY = "5b3ce3597851110001cf62485c01b629a61e4f57b5449ab3a0000ec5"

def get_temp_series(location = "Bergheimer Straße 116, 69115 Heidelberg, Germany"):
    """
    Given a location returns an xarray dataset of hourly temperatures in Kelvin
    INPUTS:
        something
    OUTPUTS:
        ds (xarray dataset): has coordinates [longitude (float32), latitude (float32), time (datetime64)]
        
    TODO: specify time bounds, properly comment
    """
    clnt = openrouteservice.Client(key=ORS_API_KEY)
    call = pelias_search(clnt, location)
    for feature in call["features"]:
        geom = feature["geometry"]["coordinates"]

    fn = "temp_data.nc"
    ds = xr.open_dataset(fn)
    #yearmonth = "2021-10-31"
    #yearmonth = datetime.strptime(yearmonth, '%Y-%m-%d')
    ds = ds.sel(latitude=round(geom[1])).sel(longitude=round(geom[0]))

    # interpolate missing hours of the day (sometimes data only from 06h to 22h)
    ds = ds.resample(time="1h").interpolate("linear")
    return ds


def mask_temp(data, Tref = 291.15):
    """
    INPUTS:
        data (xarray dataset): Must have datetimes in ["time"].values and temperatures in ["t2m"].values
        Tref (float): Reference temperature, given in Kelvin. Default is 291.15 (18 Celcius).
    OUTPUTS:
        something
    Note that for now I choose not to use heating start and stop times (ex. setting temp = Tref outside this range)
    becuase it adds complications, ex. the building will maintain some of the heating even if it is off... I think
    that turning the heating off will result in lower gas usage anyway... idk should talk about it
    """
    pass

    # hourly time series
    # compute degree days

def calc_degreedays(type, Tref=291.15):
    """ Docstring TODO """
    if type not in ('heating','cooling'):
        print("WARNING: degree days type not understood. Defaulting to type='heating'.")
        type = 'heating'
    
    ds = get_temp_series() 
    
    feb = ds.sel(time=slice('2020-02-01','2020-02-03')) # this takes from midnight of first day up until 11pm on the last day (note last hour missing)
    diff_time = feb['time'][-1] - feb['time'][0] # get time difference, saved as xarray with np.timedelta64 array in values

    if type == 'heating':
        feb['t2m'] = Tref - feb['t2m']
    else: # type == cooling
        feb['t2m'] = feb['t2m'] - Tref
    feb = np.maximum(feb, 0)

    # integrate
    # note: integrate sets each hour as unit 1, must divide by total number of hours to obtain cumulative degree days
    dd = feb.integrate('time',datetime_unit='h') / np.timedelta64(diff_time.values,'h').astype(float)
    
    return dd['t2m'].values

In [121]:
ds = get_temp_series() 
calc_degreedays('heating')

array(7.69409022)

TODO:
- be able to select date range (select months individually? Must be careful of leap years for february)
- can we do calculation for each month simultaneously for a given total range?
- look into only calculating degree days based on fixed hours of the day
- look into alternative methods of calculating degree days, ex. soft cutoff below Tref or thresholding to a few degrees below Tref